# Lab 4 - Reinforcement Learning

## Read details of the environment & model from a text file
Read: 

- Width, Height
- Noise: probability of not going in the intended direction (then divided by two for two unintentional neighbor directions)
- Immediate rewards of non-goal states
- Terminal (goal) states: their locations and rewards
- Internal Walls: locations

Create the transition matrix (T) and the Rewards matrix according to above details.

"grid1.txt" corresponds to the example in AIMA and the lecture slides. You can double-check various results there.

Python style comments in grid1.txt are just comments, can be ignored/deleted.

Task1 - We ask you to create two more grids (files) of moderate size and with at least one internal wall. And perform the analysis described below on these three grids.

In [64]:
import numpy as np
file1 = open('grid1.txt', 'r')
(W, H) = [t(s) for t,s in zip((int,int),file1.readline().split())]
noise = [t(s) for t,s in zip((float,str),file1.readline().split())][0]
immediate_rewards = [t(s) for t,s in zip((float,str),file1.readline().split())][0]
N = W*H #total number of states
NA = 4 #number of actions
T = np.zeros((N,N,NA)) #state transition probabilities
Directions = np.array(['L','R','U','D'])
Rewards = np.zeros((N,1)) + immediate_rewards #immediate rewards
Terminals = np.zeros((N,1)) #terminal states
Walls = np.zeros((N,1)) #(internal) wall states
prob1 = 1-noise
prob2 = noise/2
while True:
    line = file1.readline()
    if not line:
        break
    if line.find('T') != -1:
        (_,x, y, r) = [t(s) for t,s in zip((str,int,int,float),line.split())]
        Terminals[y*W+x] = 1
        Rewards[y*W+x] = r
    if line.find('W') != -1:
        (_,x, y) = [t(s) for t,s in zip((str,int,int),line.split())]
        Walls[y*W+x] = 1
file1.close()
for i in range(W):
    for j in range(H):
        if i+1<W and Walls[j*W+i+1]==0: #not wall on the right
            T[j*W+i,j*W+i+1,1] += prob1
            T[j*W+i,j*W+i+1,2] += prob2
            T[j*W+i,j*W+i+1,3] += prob2
        else:
            T[j*W+i,j*W+i,1] += prob1
            T[j*W+i,j*W+i,2] += prob2
            T[j*W+i,j*W+i,3] += prob2
        if i-1>=0 and Walls[j*W+i-1]==0: #not wall on the left
            T[j*W+i,j*W+i-1,0] += prob1
            T[j*W+i,j*W+i-1,2] += prob2
            T[j*W+i,j*W+i-1,3] += prob2
        else:
            T[j*W+i,j*W+i,0] += prob1
            T[j*W+i,j*W+i,2] += prob2
            T[j*W+i,j*W+i,3] += prob2
        if j+1<H and Walls[(j+1)*W+i]==0: #not wall below
            T[j*W+i,(j+1)*W+i,3] += prob1
            T[j*W+i,(j+1)*W+i,0] += prob2
            T[j*W+i,(j+1)*W+i,1] += prob2
        else:
            T[j*W+i,j*W+i,3] += prob1
            T[j*W+i,j*W+i,0] += prob2
            T[j*W+i,j*W+i,1] += prob2
        if j-1>=0 and Walls[(j-1)*W+i]==0: #not wall above
            T[j*W+i,(j-1)*W+i,2] += prob1
            T[j*W+i,(j-1)*W+i,0] += prob2
            T[j*W+i,(j-1)*W+i,1] += prob2
        else:
            T[j*W+i,j*W+i,2] += prob1
            T[j*W+i,j*W+i,0] += prob2
            T[j*W+i,j*W+i,1] += prob2

## Task 2 - Implement functions

### Value and Policy iteration
Value iteration is provided, implement policy iteration

### TD-Learning and Q-Learning
Q-Learning is implemented, implement TD-Learning

In [65]:
def value_iteration(gamma, T, Rewards, Terminals, Walls):
    N, NA = T.shape[0], T.shape[2]
    V = np.zeros((N,1))
    Policy = np.zeros((N,1))
    iteration = 0
    while True:
        iteration += 1
        V_old = np.copy(V)
        for s in range(N):
            if Walls[s]==1: continue
            if Terminals[s]==1:
                V[s] = Rewards[s]
                continue
            Q = np.zeros((NA,1))
            for a in range(NA):
                Q[a] = Rewards[s] + gamma*np.dot(T[s,:,a],V)
            V[s] = np.max(Q)
            Policy[s] = np.argmax(Q)
        if np.sum(np.abs(V-V_old))<1e-10:
            print('Value-Iteration converged at step %d.' % (iteration))
            break
    return V, Policy

def policy_iteration(gamma, T, Rewards, Terminals, Walls):
    def policy_evaluation(pi, V, T, Rewards, Terminals, Walls):
        N, NA = T.shape[0], T.shape[2]
        for s in range(N):
            if Walls[s] == 1: continue
            if Terminals[s] == 1:
                V[s] = Rewards[s]
            if (Walls[s]!=1) & (Terminals[s] != 1):
                vv = np.zeros((1, N))
                vv[0, s] = 1.0
                action = int(pi[s])
                Temp = np.dot(vv, T[:, :, action])
                V[s] = Rewards[s] + np.sum(np.multiply(gamma*V, np.transpose(Temp)))
        return V
    ######## PI####################
    iteration = 0
    N, NA = T.shape[0], T.shape[2]
    V = np.zeros((N,1))
    Policy = np.zeros((N,1))

    while True:
        iteration += 1
        # Policy evaluation
        V = policy_evaluation(Policy, V, T, Rewards, Terminals, Walls)
        Policy_old= Policy.copy()
        for s in range(N):
            Q = np.zeros((NA, 1))
            vv = np.zeros((1, N))
            vv[0, s] = 1.0
            for a in range(NA):
                Q[a] = Rewards[s] + np.sum(gamma*np.dot(T[s,:,a],V))
            aQ = np.argmax(Q)
            if aQ != Policy[s]:
                Policy[s] = aQ
        # Stopping criteria
        if (np.all(Policy == Policy_old)):
            print('Policy-Iteration converged at step %d.' % (iteration))
            break
    print('Iter:',iteration)
    return V, Policy

def TD_learning(N_episodes,epsilon, alpha, gamma, T, Rewards, Terminals, Walls):
    alpha=0.8 #0.8 grid1 #0.6 grid2,grid3
    N, NA = T.shape[0], T.shape[2]
    V = np.zeros((N,1))
    Policy = np.ones((N,1))
    #Policy = np.argmax(Policy,axis=1)
    # Implement TD
    for e in range(N_episodes):
        s = int(np.floor(np.random.uniform(0,N-1)))
        while Terminals[s]==1 or Walls[s]==1:
            s = int(np.floor(np.random.uniform(0,N-1)))
        while Terminals[s]==0:
            u = np.random.uniform(0,1)
            a =int(Policy[s])
            u = np.random.uniform(0,1)
            #temp = np.cumsum(T[s, :, a])
            s1 = np.argmax(u<np.cumsum(T[s,:,a]))

            if (V[s1]==0):
                V[s1] = Rewards[s1]
            elif Terminals[s]==1:
                V[s] = Rewards[s]
            else:
                V[s] += alpha * (Rewards[s] + (gamma * V[s1])-V[s])

            s = s1
    V[Terminals==1] = Rewards[Terminals==1]
    return V, Policy

    # utils

def Q_learning(N_episodes, epsilon, alpha, gamma, T, Rewards, Terminals, Walls):
    N, NA = T.shape[0], T.shape[2]
    Q = np.zeros((N,NA))
    for e in range(N_episodes):
        s = int(np.floor(np.random.uniform(0,N-1)))
        while Terminals[s]==1 or Walls[s]==1:
            s = int(np.floor(np.random.uniform(0,N-1)))
        while Terminals[s]==0:
            u = np.random.uniform(0,1)
            if u<epsilon:
                a = int(np.floor(np.random.uniform(0,NA)))
            else:
                a = np.argmax(Q[s,:])
            u = np.random.uniform(0,1)
            temp=np.cumsum(T[s,:,a])
            s1 = np.argmax(u<np.cumsum(T[s,:,a]))
            Q[s,a] += alpha*(Rewards[s1] + gamma*np.max(Q[s1,:]) - Q[s,a])
            s = s1
        epsilon = epsilon*0.9999 # Modify here for other annealing regimes
        alpha = alpha*0.9999 # Modify here for other annealing regimes
    V = np.max(Q,axis=1)
    V = V[:,None]
    V[Terminals==1] = Rewards[Terminals==1]
    Policy = np.array(np.argmax(Q,axis=1)).reshape((N,1))
    return V, Policy

## Task 3 - Experiments

For the three grids you have, perform the following analysis and include them in your report,.

- Compare value and policy iterations in terms of convergence time. Relate it to computational complexity, current implementation and number of iterations needed.

- How are optimal policies change with immediate reward values? Show some examples (similar to Figure 17.2b in AIMA)

- Compare TD-Learning and Q-Learning results with each other. Also with value and policy iterations. Remember that value and policy iteration solve Markov Decision Processes where we know the model (T and Rewards). TD-Learning and Q-Learning are (passive and active, respectively) Reinforcement Learning methods that don't have the model but use data from simulations. Data are simulated from the model we know, but the model is not used n TD or Q-Learning.

- What are the effects of epsilon and alpha values and how they are modified?

- What is the effect of number of episodes?

In [66]:
gamma = 0.9
alpha = 0.9
epsilon = 0.9
N_episodes_TD = 60000
N_episodes_Q = 20000

#Value iteration
print('####################### Value Iteration ########################################')
V_VI, Policy_VI = value_iteration(gamma, T, Rewards, Terminals, Walls)
print(V_VI.reshape((H,W)))
maze = Directions[Policy_VI.astype(int)]
maze[Walls==1] = 'W'
maze[Terminals==1] = 'G'
print(maze.reshape((H,W)))
#Policy iteration
print('########################## Policy Iteration ####################################')
V_PI, Policy_PI = policy_iteration(gamma, T, Rewards, Terminals, Walls)
print(V_PI.reshape((H,W)))
maze_PI = Directions[Policy_PI.astype(int)]
maze_PI[Walls==1] = 'W'
maze_PI[Terminals==1] = 'G'
print(maze_PI.reshape((H,W)))
#
#TD-Learning
print('################################ TD Learning ###################################')
V_TD, Policy_TD = TD_learning(N_episodes_TD,epsilon, alpha, gamma, T, Rewards, Terminals, Walls)
print(V_TD.reshape((H,W)))
maze_TD = Directions[Policy_TD.astype(int)]
maze_TD[Walls==1] = 'W'
maze_TD[Terminals==1] = 'G'
print(maze_TD.reshape((H,W)))

#Q-Learning
print('######################### Q Learning ###########################################')
V_Q, Policy_Q = Q_learning(N_episodes_Q, epsilon, alpha, gamma, T, Rewards, Terminals, Walls)

print(V_Q.reshape((H,W)))
maze_Q = Directions[Policy_Q.astype(int)]
maze_Q[Walls==1] = 'W'
maze_Q[Terminals==1] = 'G'
print(maze_Q.reshape((H,W)))

#

####################### Value Iteration ########################################
Value-Iteration converged at step 24.
[[ 0.5094156   0.64958636  0.79536224  1.        ]
 [ 0.39851125  0.          0.48644046 -1.        ]
 [ 0.29646654  0.25396055  0.3447884   0.12994247]]
[['R' 'R' 'R' 'G']
 ['U' 'W' 'U' 'G']
 ['U' 'R' 'U' 'L']]
########################## Policy Iteration ####################################
Policy-Iteration converged at step 5.
Iter: 5
[[ 0.39984038  0.62539304  0.79264275  1.        ]
 [ 0.26742185  0.          0.48293355 -1.        ]
 [ 0.14895235  0.196516    0.33360253  0.11839775]]
[['R' 'R' 'R' 'G']
 ['U' 'W' 'U' 'G']
 ['U' 'R' 'U' 'L']]
################################ TD Learning ###################################
[[ 0.46807757  0.69004957  0.85954784  1.        ]
 [ 0.4170298   0.         -0.93890383 -1.        ]
 [-0.73277916 -0.82120339 -0.81908491 -0.87166701]]
[['R' 'R' 'R' 'G']
 ['R' 'W' 'R' 'G']
 ['R' 'R' 'R' 'R']]
######################### Q Learning 

## Task 4 (extra credit) - What if diagonal moves were possible?

Repeat the experiments in one of these settings. You don't have to stick to the code provided here. But, you can only use Numpy (and matplotlib).

You should decide on how to distribute the noise associated with actions across (next) states. For example, in the scenario above we have probability of going Up with the Up action is 1-noise. The probability of going Left with the Up action is noise/2. The same for going Right.

In [67]:
import numpy as np
file1 = open('grid1.txt', 'r')
(W, H) = [t(s) for t,s in zip((int,int),file1.readline().split())]
noise = [t(s) for t,s in zip((float,str),file1.readline().split())][0]
immediate_rewards = [t(s) for t,s in zip((float,str),file1.readline().split())][0]
N = W*H #total number of states
NA = 8 #number of actions
T = np.zeros((N,N,NA)) #state transition probabilities
Directions = np.array(['L','R','U','D','UL','UR','DL','DR'])
Rewards = np.zeros((N,1)) + immediate_rewards #immediate rewards
Terminals = np.zeros((N,1)) #terminal states
Walls = np.zeros((N,1)) #(internal) wall states
prob1 = 1-noise #noise
prob2 = noise/2 #noise/2
while True:
    line = file1.readline()
    if not line:
        break
    if line.find('T') != -1:
        (_,x, y, r) = [t(s) for t,s in zip((str,int,int,float),line.split())]
        Terminals[y*W+x] = 1
        Rewards[y*W+x] = r
    if line.find('W') != -1:
        (_,x, y) = [t(s) for t,s in zip((str,int,int),line.split())]
        Walls[y*W+x] = 1
file1.close()
for i in range(W):
    for j in range(H):
        if i+1<W and Walls[j*W+i+1]==0: #not wall on the right
           T[j*W+i,j*W+i+1,1] += prob1
           T[j*W+i, j*W+i+1,5] += prob2
           T[j*W+i, j*W+i+1,7] += prob2
        else:
            T[j*W+i, j*W+i, 1] += prob1
            T[j*W+i, j*W+i, 5] += prob2
            T[j*W+i, j*W+i, 7] += prob2

        if i-1>=0 and Walls[j*W+i-1]==0: #not wall on the left
            T[j*W+i,j*W+i-1,0] += prob1
            T[j*W+i,j*W+i-1,4] += prob2
            T[j*W+i,j*W+i-1,6] += prob2
        else:
            T[j*W+i,j*W+i,0] += prob1
            T[j*W+i,j*W+i,4] += prob2
            T[j*W+i,j*W+i,6] += prob2

        if j+1<H and Walls[(j+1)*W+i]==0: #not wall below
            T[j*W+i,(j+1)*W+i,3] += prob1
            T[j*W+i,(j+1)*W+i,6] += prob2
            T[j*W+i,(j+1)*W+i,7] += prob2
        else:
            T[j*W+i,j*W+i,3] += prob1
            T[j*W+i,j*W+i,6] += prob2
            T[j*W+i,j*W+i,7] += prob2

        if j-1>=0 and Walls[(j-1)*W+i]==0: #not wall above
            T[j*W+i,(j-1)*W+i,2] += prob1
            T[j*W+i,(j-1)*W+i,4] += prob2
            T[j*W+i,(j-1)*W+i,5] += prob2
        else:
            T[j*W+i,j*W+i,2] += prob1
            T[j*W+i,j*W+i,4] += prob2
            T[j*W+i,j*W+i,5] += prob2
        if j-1>=0 and i+1<W and Walls[(j - 1) * W + i+1] == 0:  # not wall upright
            T[j * W + i, (j-1) * W + i + 1, 5] += prob1
            T[j * W + i, (j-1) * W + i + 1, 2] += prob2
            T[j * W + i, (j-1) * W + i + 1, 1] += prob2
        else:
            T[j * W + i, j * W + i, 5] += prob1
            T[j * W + i, j * W + i, 2] += prob2
            T[j * W + i, j * W + i, 1] += prob2

        if i-1>=0 and j-1>=0  and Walls[(j - 1) * W + i-1] == 0:  # not wall upleft
            T[j * W + i, (j-1) * W + i - 1, 4] += prob1
            T[j * W + i, (j-1) * W + i - 1, 2] += prob2
            T[j * W + i, (j-1) * W + i - 1, 0] += prob2
        else:
            T[j * W + i, j * W + i, 4] += prob1
            T[j * W + i, j * W + i, 2] += prob2
            T[j * W + i, j * W + i, 0] += prob2

        if i-1>=0 and j+1<H and Walls[(j + 1) * W + i-1] == 0: # not wall downleft
            T[j * W + i, (j+1) * W + i - 1, 6] += prob1
            T[j * W + i, (j+1) * W + i - 1, 0] += prob2
            T[j * W + i, (j+1) * W + i - 1, 3] += prob2
        else:
            T[j * W + i, j * W + i, 6] += prob1
            T[j * W + i, j * W + i, 0] += prob2
            T[j * W + i, j * W + i, 3] += prob2
        if j+1<H and i+1<W and Walls[(j + 1) * W + i+1] == 0: # not wall downright
            T[j * W + i, (j+1) * W + i + 1, 7] += prob1
            T[j * W + i, (j+1) * W + i + 1, 1] += prob2
            T[j * W + i, (j+1) * W + i + 1, 3] += prob2
        else:
            T[j * W + i, j * W + i, 7] += prob1
            T[j * W + i, j * W + i, 1] += prob2
            T[j * W + i, j * W + i, 3] += prob2

gamma = 0.9

#Value iteration
print('####################### Value Iteration ########################################')
V_VI, Policy_VI = value_iteration(gamma, T, Rewards, Terminals, Walls)
print(V_VI.reshape((H,W)))
maze = Directions[Policy_VI.astype(int)]
maze[Walls==1] = 'W'
maze[Terminals==1] = 'G'
print(maze.reshape((H,W)))
#Policy iteration
print('########################## Policy Iteration ####################################')
V_PI, Policy_PI = policy_iteration(gamma, T, Rewards, Terminals, Walls)
print(V_PI.reshape((H,W)))
maze_PI = Directions[Policy_PI.astype(int)]
maze_PI[Walls==1] = 'W'
maze_PI[Terminals==1] = 'G'
print(maze_PI.reshape((H,W)))
#


####################### Value Iteration ########################################
Value-Iteration converged at step 19.
[[ 0.41934961  0.53314817  0.64835165  1.        ]
 [ 0.41934961  0.          0.64835165 -1.        ]
 [ 0.3975719   0.50834578  0.3975719   0.37259466]]
[['R' 'R' 'R' 'G']
 ['UR' 'W' 'UR' 'G']
 ['R' 'UR' 'L' 'UL']]
########################## Policy Iteration ####################################
Policy-Iteration converged at step 5.
Iter: 5
[[ 0.38842938  0.53243351  0.64830324  1.        ]
 [ 0.41015875  0.          0.64834729 -1.        ]
 [ 0.3562397   0.50406158  0.38879859  0.37180192]]
[['R' 'DR' 'R' 'G']
 ['UR' 'W' 'UR' 'G']
 ['R' 'UR' 'L' 'UL']]
